In [11]:
# Cài đặt thư viện nếu thiếu
# %pip install pandas numpy scikit-learn pyarrow

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
from sklearn.preprocessing import LabelEncoder

# Tắt cảnh báo đỏ cho gọn màn hình
warnings.filterwarnings('ignore')

print("✅ Bước 1: Đã nạp thư viện thành công!")

✅ Bước 1: Đã nạp thư viện thành công!


In [12]:
def load_data_optimized(file_path):
    print(f"🔄 Đang đọc file '{file_path}'...")
    
    # Định nghĩa kiểu dữ liệu để máy không phải đoán (Tăng tốc độ)
    column_types = {
        'manufacturer': 'string',
        'model': 'string',
        'year': 'int32',
        'mileage': 'float32',
        'price': 'float32',
        'mpg': 'string',
        'fuel_type': 'string',  
        'engine': 'string',
        'accidents_or_damage': 'float32', # 1.0 = Có, 0.0 = Không
        'one_owner': 'float32',           # 1.0 = Một chủ, 0.0 = Nhiều chủ
        'driver_rating': 'float32'
    }
    
    # Chỉ đọc những cột cần thiết
    cols = list(column_types.keys())

    try:
        # Ưu tiên dùng engine PyArrow (nhanh nhất)
        df = pd.read_csv(file_path, usecols=cols, dtype=column_types, engine='pyarrow')
    except:
        try:
            # Fallback về engine thường nếu máy chưa cài pyarrow
            df = pd.read_csv(file_path, usecols=cols, dtype=column_types)
        except FileNotFoundError:
            print("⚠️ Không tìm thấy file csv. Đang tạo dữ liệu giả lập...")
            # return pd.DataFrame({
            #     'manufacturer': ['Toyota', 'Ford', 'Honda', 'BMW', 'Lexus'] * 100,
            #     'model': ['Camry', 'F-150', 'Civic', '320i', 'RX350'] * 100,
            #     'year': [2015, 2018, 2012, 2010, 2021] * 100,
            #     'mileage': [40000, 30000, 120000, 150000, 10000] * 100,
            #     'price': [15000, 25000, 8000, 9000, 45000] * 100,
            #     'mpg': ['30-35', '18-24', '32-40', '25-30', '20-25'] * 100,
            #     'fuel_type': ['Hybrid', 'Gasoline', 'Gasoline', 'Diesel', 'Electric'] * 100, 
            #     'engine': ['2.5L 4-Cyl', '5.0L V8', '1.8L 4-Cyl', '2.0L Turbo', 'Electric Motor'] * 100, 
            #     'accidents_or_damage': [0, 0, 1, 1, 0] * 100,
            #     'one_owner': [1, 1, 0, 0, 1] * 100,
            #     'driver_rating': [4.5, 4.8, 3.5, 3.8, 5.0] * 100
            # })

    # Xử lý sơ bộ: Điền giá trị thiếu (NaN)
    df = df.fillna(0) 
    print(f"✅ Bước 2: Đã tải xong {len(df):,} dòng dữ liệu.")
    return df

# Tải dữ liệu
df_raw = load_data_optimized('cars.csv')

🔄 Đang đọc file 'cars.csv'...
✅ Bước 2: Đã tải xong 762,091 dòng dữ liệu.


In [13]:
print("🤖 Đang huấn luyện AI 'Chuyên gia rủi ro'...")

# 1. Tạo nhãn Rủi ro giả lập để dạy học
def create_risk_label(row):
    score = 0
    if row['accidents_or_damage'] == 1: score += 3 
    if row['year'] < 2014: score += 1              
    if row['mileage'] > 120000: score += 2         
    if row['one_owner'] == 0: score += 1           
    
    if score <= 1: return 'Low'
    elif score <= 3: return 'Medium'
    else: return 'High'

# Lấy mẫu để train cho nhanh
df_train = df_raw.sample(min(50000, len(df_raw))).copy() 
df_train['Risk_Level'] = df_train.apply(create_risk_label, axis=1)

# 2. Huấn luyện Random Forest
features = ['year', 'mileage', 'accidents_or_damage', 'one_owner', 'driver_rating']
X = df_train[features]
y = df_train['Risk_Level']

clf = RandomForestClassifier(n_estimators=50, random_state=42)
clf.fit(X, y)

print(f"✅ Bước 2: AI đã học xong!")

🤖 Đang huấn luyện AI 'Chuyên gia rủi ro'...
✅ Bước 2: AI đã học xong!


In [14]:
# =======================================================
# BẢNG CẤU HÌNH SO SÁNH CẶP (PAIRWISE COMPARISON)
# =======================================================

# 1. So sánh GIÁ (Price)
PRICE_vs_YEAR      = 3      
PRICE_vs_MILEAGE   = 2      
PRICE_vs_MPG       = 5      
PRICE_vs_ACCIDENTS = 1      
PRICE_vs_BRAND     = 2      
PRICE_vs_FUEL      = 3      
PRICE_vs_ENGINE    = 5      

# 2. So sánh NĂM (Year)
YEAR_vs_MILEAGE    = 1/2    
YEAR_vs_MPG        = 2      
YEAR_vs_ACCIDENTS  = 1/3    
YEAR_vs_BRAND      = 1      
YEAR_vs_FUEL       = 2      
YEAR_vs_ENGINE     = 3      

# 3. So sánh SỐ KM (Mileage)
MILEAGE_vs_MPG       = 3    
MILEAGE_vs_ACCIDENTS = 1/2  
MILEAGE_vs_BRAND     = 2    
MILEAGE_vs_FUEL      = 2    
MILEAGE_vs_ENGINE    = 3

# 4. So sánh XĂNG (MPG) 
MPG_vs_ACCIDENTS     = 1/5  
MPG_vs_BRAND         = 1/2  
MPG_vs_FUEL          = 1    
MPG_vs_ENGINE        = 2    

# 5. TAI NẠN (Accidents)
ACCIDENTS_vs_BRAND   = 3    
ACCIDENTS_vs_FUEL    = 3    
ACCIDENTS_vs_ENGINE  = 5    

# 6. THƯƠNG HIỆU (Brand)
BRAND_vs_FUEL        = 2    
BRAND_vs_ENGINE      = 3    

# 7. NHIÊN LIỆU (Fuel)
FUEL_vs_ENGINE       = 2    

def get_ahp_weights():
    # Thứ tự quan trọng: [0:Price, 1:Year, 2:Mileage, 3:MPG, 4:Accidents, 5:Brand, 6:Fuel, 7:Engine]
    n = 8
    matrix = np.ones((n, n))
    
    def set_val(i, j, val):
        matrix[i, j] = val
        matrix[j, i] = 1 / val

    # Điền ma trận
    set_val(0, 1, PRICE_vs_YEAR); set_val(0, 2, PRICE_vs_MILEAGE); set_val(0, 3, PRICE_vs_MPG)
    set_val(0, 4, PRICE_vs_ACCIDENTS); set_val(0, 5, PRICE_vs_BRAND); set_val(0, 6, PRICE_vs_FUEL); set_val(0, 7, PRICE_vs_ENGINE)
    
    set_val(1, 2, YEAR_vs_MILEAGE); set_val(1, 3, YEAR_vs_MPG); set_val(1, 4, YEAR_vs_ACCIDENTS)
    set_val(1, 5, YEAR_vs_BRAND); set_val(1, 6, YEAR_vs_FUEL); set_val(1, 7, YEAR_vs_ENGINE)
    
    set_val(2, 3, MILEAGE_vs_MPG); set_val(2, 4, MILEAGE_vs_ACCIDENTS); set_val(2, 5, MILEAGE_vs_BRAND)
    set_val(2, 6, MILEAGE_vs_FUEL); set_val(2, 7, MILEAGE_vs_ENGINE)
    
    set_val(3, 4, MPG_vs_ACCIDENTS); set_val(3, 5, MPG_vs_BRAND); set_val(3, 6, MPG_vs_FUEL); set_val(3, 7, MPG_vs_ENGINE)

    set_val(4, 5, ACCIDENTS_vs_BRAND); set_val(4, 6, ACCIDENTS_vs_FUEL); set_val(4, 7, ACCIDENTS_vs_ENGINE)
    
    set_val(5, 6, BRAND_vs_FUEL); set_val(5, 7, BRAND_vs_ENGINE)
    
    set_val(6, 7, FUEL_vs_ENGINE)
    
    # Tính trọng số
    weights = (matrix / matrix.sum(axis=0)).mean(axis=1)
    return weights

weights = get_ahp_weights()
print("✅ Bước 3: Đã tính toán xong bộ trọng số AHP!")
criteria_names = ['Price', 'Year', 'Mileage', 'MPG', 'Accidents', 'Brand', 'Fuel', 'Engine']
for n, w in zip(criteria_names, weights):
    print(f"   - {n}: {w*100:.2f}%")

✅ Bước 3: Đã tính toán xong bộ trọng số AHP!
   - Price: 23.89%
   - Year: 9.85%
   - Mileage: 14.72%
   - MPG: 5.54%
   - Accidents: 25.10%
   - Brand: 10.36%
   - Fuel: 6.67%
   - Engine: 3.88%


In [15]:
def process_final_decision(df, weights, ai_model):
    df = df.copy()
    
    # --- 1. XỬ LÝ SỐ LIỆU MPG ---
    def parse_mpg(val):
        try:
            s = str(val)
            if '-' in s: return (float(s.split('-')[0]) + float(s.split('-')[1])) / 2
            return float(s)
        except: return 0
    df['mpg_num'] = df['mpg'].apply(parse_mpg)

    # --- 2. CHUYỂN ĐỔI CÁC CỘT CHỮ SANG SỐ (SCORING) ---
    
    # A. Chấm điểm Nhiên liệu (Fuel Score)
    # Ưu tiên xe điện/lai xăng điện > Xăng > Dầu
    def score_fuel(val):
        val = str(val).lower()
        if 'electric' in val or 'hybrid' in val or 'plug-in' in val: return 1.0 # Tốt nhất
        if 'gasoline' in val or 'flex' in val: return 0.5   # Trung bình
        if 'diesel' in val: return 0.3                      # Thấp
        return 0.5
    df['fuel_score'] = df['fuel_type'].apply(score_fuel)

    # B. Chấm điểm Thương hiệu (Brand Score)
    # Cách đơn giản: Dùng LabelEncoder để chuyển tên hãng thành số (hoặc dùng giá trị trung bình)
    # Ở đây tôi dùng "Tần suất xuất hiện" làm thước đo độ phổ biến (Uy tín)
    brand_counts = df['manufacturer'].value_counts(normalize=True)
    df['brand_score'] = df['manufacturer'].map(brand_counts)

    # C. Chấm điểm Động cơ (Engine Score)
    # Vì động cơ rất phức tạp, ta tạm thời dùng LabelEncoder để mã hóa
    le = LabelEncoder()
    df['engine_score'] = le.fit_transform(df['engine'].astype(str))
    # Chuẩn hóa về 0-1 ngay
    df['engine_score'] = (df['engine_score'] - df['engine_score'].min()) / (df['engine_score'].max() - df['engine_score'].min())

    # --- 3. CHUẨN HÓA DỮ LIỆU (0-1) ---
    df_norm = pd.DataFrame()
    
    # Nhóm Cost (Càng thấp càng tốt)
    for col in ['price', 'mileage', 'accidents_or_damage']:
        val = df[col].astype(float)
        if val.max() != val.min():
            df_norm[col] = (val.max() - val) / (val.max() - val.min())
        else: df_norm[col] = 0

    # Nhóm Benefit (Càng cao càng tốt)
    # Lưu ý: mpg, brand, fuel, engine, year đều là càng cao càng tốt
    for col_raw, col_norm in [('year', 'year'), ('mpg_num', 'mpg'), 
                              ('brand_score', 'brand'), ('fuel_score', 'fuel'), ('engine_score', 'engine')]:
        val = df[col_raw].astype(float)
        if val.max() != val.min():
            df_norm[col_norm] = (val - val.min()) / (val.max() - val.min())
        else: df_norm[col_norm] = 0
    
    # --- 4. TÍNH ĐIỂM AHP TỔNG HỢP (FULL 8 TIÊU CHÍ) ---
    # weights index: [0:Price, 1:Year, 2:Mileage, 3:MPG, 4:Accidents, 5:Brand, 6:Fuel, 7:Engine]
    df['AHP_Score'] = (
        df_norm['price'] * weights[0] +
        df_norm['year'] * weights[1] +
        df_norm['mileage'] * weights[2] +
        df_norm['mpg'] * weights[3] +
        df_norm['accidents_or_damage'] * weights[4] +
        df_norm['brand'] * weights[5] +
        df_norm['fuel'] * weights[6] +
        df_norm['engine'] * weights[7]
    )
    
    # --- 5. KẾT HỢP AI DỰ BÁO ---
    # Lấy top 50 xe điểm AHP cao nhất để chạy AI cho nhẹ máy
    top_cars = df.sort_values(by='AHP_Score', ascending=False).head(50).copy()
    
    ai_features = ['year', 'mileage', 'accidents_or_damage', 'one_owner', 'driver_rating']
    top_cars['Predicted_Risk'] = ai_model.predict(top_cars[ai_features])
    
    return top_cars

# Chạy tính toán
final_result = process_final_decision(df_raw, weights, clf)

# --- HIỂN THỊ KẾT QUẢ ---
print("\n" + "="*80)
print("🏆 TOP 10 XE TỐI ƯU NHẤT (AHP Score cao nhất & Có AI cảnh báo rủi ro)")
print("="*80)

# Chọn cột hiển thị
cols = ['manufacturer', 'model', 'year', 'price', 'mileage', 'fuel_type', 'AHP_Score', 'Predicted_Risk']

# Tô màu rủi ro (Chỉ hoạt động trên Jupyter Notebook)
try:
    def highlight_risk(val):
        color = 'red' if val == 'High' else ('green' if val == 'Low' else 'orange')
        return f'color: {color}; font-weight: bold'
    
    display(final_result[cols].head(10).style.applymap(highlight_risk, subset=['Predicted_Risk']))
except:
    print(final_result[cols].head(10).to_string(index=False))


🏆 TOP 10 XE TỐI ƯU NHẤT (AHP Score cao nhất & Có AI cảnh báo rủi ro)
manufacturer            model  year   price  mileage fuel_type  AHP_Score Predicted_Risk
        Ford         Maverick  2023 42971.0    507.0    Hybrid   0.954786            Low
        Ford  Maverick LARIAT  2023 44971.0    768.0    Hybrid   0.954751            Low
        Ford     Maverick XLT  2022 30788.0  10493.0    Hybrid   0.953007            Low
        Ford     Maverick XLT  2022 35686.0  11050.0    Hybrid   0.952932            Low
        Ford Fusion Hybrid SE  2018 21995.0  45070.0    Hybrid   0.946797            Low
        Ford Fusion Hybrid SE  2018 21417.0  48106.0    Hybrid   0.946397            Low
        Ford F-150 King Ranch  2021 63998.0  21226.0    Hybrid   0.945999            Low
        Ford     F-150 Lariat  2021 53726.0  37000.0    Hybrid   0.945939            Low
        Ford C-Max Energi SEL  2016 18598.0  32600.0    Hybrid   0.944893            Low
        Ford Fusion Hybrid SE  2019 1779